# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [91]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
import json
# Import API key
from api_keys import geoapify_key

In [105]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,adamstown,-25.0660,-130.1015,22.67,85,100,6.84,PN,1712791119
1,1,port-aux-francais,-49.3500,70.2167,7.44,96,100,3.79,TF,1712791120
2,2,midvagur,62.0511,-7.1939,6.46,87,100,12.35,FO,1712791020
3,3,smithers,54.7804,-127.1743,8.18,46,100,5.66,CA,1712791590
4,4,haiku-pauwela,20.9219,-156.3051,29.01,62,20,8.23,US,1712791590


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [93]:
%%capture --no-display

# Configure the map plot
map_plot_1 = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "EsriImagery",
    scale = "Humidity",
    color = "City",
    frame_width = 800,
    frame_height = 600    
)


# Display the map
map_plot_1

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [94]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_lst = []
for x in city_data_df["Max Temp"]:
    if  x > 21:
        if x < 27:
            ideal_lst.append(x)
    

city_data_ideal = pd.DataFrame(ideal_lst)
city_data_ideal = city_data_ideal.rename(columns={0: 'Max Temp'})
city_data_ideal = pd.merge(city_data_ideal, city_data_df)
city_data_ideal = city_data_ideal.drop(city_data_ideal[city_data_ideal['Wind Speed'] > 4.5].index)
city_data_ideal = city_data_ideal.drop(city_data_ideal[city_data_ideal["Cloudiness"]>0].index)
# Drop any rows with null values
# YOUR CODE HERE
city_data_ideal = city_data_ideal.dropna()

In [95]:
# Display sample data

city_data_ideal

,Max Temp,City_ID,City,Lat,Lng,Humidity,Cloudiness,Wind Speed,Country,Date
9,25.91,50,mukalla,14.5425,49.1242,75,0,1.80,YE,1712791604
101,25.40,198,arraial do cabo,-22.9661,-42.0278,87,0,4.41,BR,1712790953
136,26.06,292,sittwe,20.1500,92.9000,64,0,3.63,MM,1712791678
138,26.06,292,sittwe,20.1500,92.9000,64,0,3.63,MM,1712791678
150,22.89,312,urangan,-25.3000,152.9000,65,0,4.47,AU,1712791685
151,25.05,315,sao joao da barra,-21.6403,-41.0511,86,0,3.94,BR,1712791158


### Step 3: Create a new DataFrame called `hotel_df`.

In [96]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = city_data_ideal
hotel_df = hotel_df.drop(labels=["Max Temp","City_ID","Cloudiness","Wind Speed", "Date"],axis=1)
hotel_df = hotel_df.loc[:,["City", "Country", "Lat", "Lng", "Humidity"]]
# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ''

# Display sample data
hotel_df


,City,Country,Lat,Lng,Humidity,Hotel Name
9,mukalla,YE,14.5425,49.1242,75,
101,arraial do cabo,BR,-22.9661,-42.0278,87,
136,sittwe,MM,20.1500,92.9000,64,
138,sittwe,MM,20.1500,92.9000,64,
150,urangan,AU,-25.3000,152.9000,65,
151,sao joao da barra,BR,-21.6403,-41.0511,86,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [103]:
# Set parameters to search for a hotel
radius = 10000
categories = "accommodation.hotel"


limit = 20


params = {
    "categories":categories,
    "limit":limit,
    "filter":filters,
    "bias":bias,
    "apiKey":geoapify_key
}

print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = hotel_df["Lat"]
    longitude = hotel_df["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = "https://api.geoapify.com/v2/places?PARAMS"
    print(name_address)


Starting hotel search
https://api.geoapify.com/v2/places?PARAMS
https://api.geoapify.com/v2/places?PARAMS
https://api.geoapify.com/v2/places?PARAMS
https://api.geoapify.com/v2/places?PARAMS
https://api.geoapify.com/v2/places?PARAMS
https://api.geoapify.com/v2/places?PARAMS


In [78]:
# Set parameters to search for a hotel
radius = 10000
categories = "accommodation.hotel"

filters = f"circle:{longitude},{latitude},{radius}"
bias = f"proximity:{longitude},{latitude}"
limit = 20


params = {
    "categories":categories,
    "limit":limit,
    "filter":filters,
    "bias":bias,
    "apiKey":geoapify_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = hotel_df["Lat"]
    longitude = hotel_df["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = # YOUR CODE HERE
    params["bias"] = # YOUR CODE HERE
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = # YOUR CODE HERE
    
    # Convert the API response to JSON format
    name_address = # YOUR CODE HERE
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

SyntaxError: invalid syntax (2836116758.py, line 27)

### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [ ]:
%%capture --no-display

# Configure the map plot
# YOUR CODE HERE

# Display the map
# YOUR CODE HERE